In [6]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from tqdm import trange
import time
from get_dataset_info import *
import cyvlfeat as vl

# %load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [2]:
# P1 = cv.get_canonical_camera()
# best_P2 = None
# best_X = None
# max_inliers = 0
# if epsilon > epsilon_E:

#     # P2_arr = cv.extract_P_from_E(E)
#     # X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm, x2_norm)
#     # P2_valid, X_valid = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr)
#     # valid_inliers = compute_valid_inliers(P1, P2_valid, X_valid, inliers)
#     # n_valid_inliers = np.sum(valid_inliers)
#     # print(n_valid_inliers, n_inliers)

#     # best_P2 = np.copy(P2_valid)
#     # best_X = np.copy(X_valid)
#     best_E = np.copy(E)
#     best_inliers = np.copy(inliers)
#     epsilon_E = epsilon
#     print('No. inliers:', n_inliers, end='\r')

#     # if n_valid_inliers > max_inliers:
#     #     best_inliers = np.copy(valid_inliers)
#     #     best_P2 = np.copy(P2_valid)
#     #     max_inliers = n_valid_inliers
#     #     print('No. valid inliers:', n_valid_inliers, end='\r')

#         # epsilon = max_inliers / n_points
#         # T = cv.compute_ransac_iterations(alpha, epsilon, n_samples)
#         # print('New T:', T, 'New epsilon:', epsilon)
#         # if t >= 4*T-1:
#         #     print('Bailout at iteration:', t, T)
#         #     break

# valid_inliers = compute_valid_inliers(P1, best_P2, best_X, best_inliers)
# n_valid_inliers = np.sum(valid_inliers)
# print('No. valid inliers:', n_valid_inliers, 'No. inliers:', np.sum(best_inliers))


In [32]:
def plot_3D_points(X):
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot(X[0], X[1], X[2], '.', ms=1, color='magenta', label='X')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_box_aspect([1, 1, 1]) 
    # ax.set_box_aspect((np.ptp(X[0]), np.ptp(X[1]), np.ptp(X[2])))  
    # ax.set_aspect('equal')
    # ax.view_init(elev=-50, azim=-104, roll=20)
    ax.legend(loc="lower right")
    fig.tight_layout()
    plt.show()

In [8]:
def compute_E_validity(E):
    rank = LA.matrix_rank(E)
    valid = True if rank == 2 else False
    return valid

def compute_E_inliers(E, x1_norm, x2_norm, err_threshold):
    
    distance1_arr, distance2_arr = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
    inliers = ((distance1_arr**2 + distance2_arr**2) / 2) < err_threshold**2
    n_inliers = np.sum(inliers)
    epsilon_E = n_inliers / x1_norm.shape[1]

    return epsilon_E, inliers

def verbose_E_robust(t, T_E, T_H, epsilon_E, epsilon_H, inliers, method):
    print('Iteration:', t, 'T_E:', T_E, 'T_H:', T_H, 'epsilon_E:', np.round(epsilon_E, 2), 'epsilon_H:', np.round(epsilon_H, 2), 'No. inliers:', np.sum(inliers), 'From:', method)

def compute_valid_inliers(P1, P2, X, inliers):

    x1_norm_valid = P1 @ X
    x2_norm_valid = P2 @ X
    valid_coords_P1 = x1_norm_valid[-1,:] > 0
    valid_coords_P2 = x2_norm_valid[-1,:] > 0
    valid_coords = valid_coords_P1 * valid_coords_P2
    valid_inliers = inliers * valid_coords

    return valid_inliers

def estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, err_threshold_px, verbose=False):
    
    err_threshold = err_threshold_px / K[0,0]
    best_E = None
    best_inliers = None
    n_points = x1_norm.shape[1]
    n_E_samples = 8
    n_H_samples = 4
    best_epsilon_E = 0
    best_epsilon_H = 0
    T_E = max_its
    T_H = max_its

    t = 0
    while t < T_E and t < T_H:
        t += 1

        rand_mask = np.random.choice(n_points, n_E_samples, replace=False)
        E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)
        E_valid = compute_E_validity(E)

        if E_valid:
            epsilon_E, inliers = compute_E_inliers(E, x1_norm, x2_norm, err_threshold)
                
            if epsilon_E > best_epsilon_E:
                best_E = np.copy(E)
                best_inliers = np.copy(inliers)
                best_epsilon_E = epsilon_E
                T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)

                if verbose:
                    verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='E 8-point alg.')
        
        rand_mask = np.random.choice(n_points, n_H_samples, replace=False)
        H = cv.estimate_H_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], verbose=False)
        x2_norm_proj = cv.dehomogenize(H @ x1_norm)
        distance_arr = cv.compute_point_point_distance(x2_norm_proj, x2_norm)
        inliers = distance_arr**2 < err_threshold**2
        n_inliers = np.sum(inliers)
        epsilon_H = n_inliers / n_points

        if epsilon_H > best_epsilon_H:
            
            # num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, np.eye(3))
            R1, T1, R2, T2 = cv.homography_to_RT(H, x1_norm, x2_norm)
            E1 = cv.compute_E_from_R_and_T(R1, T1)
            E2 = cv.compute_E_from_R_and_T(R2, T2)

            E1_valid = compute_E_validity(E1)
            E2_valid = compute_E_validity(E2)

            if E1_valid:
                epsilon_E, inliers = compute_E_inliers(E1, x1_norm, x2_norm, err_threshold)
                    
                if epsilon_E > best_epsilon_E:
                    best_E = np.copy(E1)
                    best_inliers = np.copy(inliers)
                    best_epsilon_E = epsilon_E
                    best_epsilon_H = epsilon_H
                    T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)
                    T_H = cv.compute_ransac_iterations(alpha, best_epsilon_H, n_H_samples, min_its, max_its, scale_its)

                    if verbose:
                        verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='H 4-point alg.')

            if E2_valid:
                epsilon_E, inliers = compute_E_inliers(E2, x1_norm, x2_norm, err_threshold)
                    
                if epsilon_E > best_epsilon_E:
                    best_E = np.copy(E2)
                    best_inliers = np.copy(inliers)
                    best_epsilon_E = epsilon_E
                    best_epsilon_H = epsilon_H
                    T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)
                    T_H = cv.compute_ransac_iterations(alpha, best_epsilon_H, n_H_samples, min_its, max_its, scale_its)
                    
                    if verbose:
                        verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='H 4-point alg.')
    
    print('Bailout at iteration:', t)
    return best_E, best_inliers

In [9]:
def compute_sift_points(img1, img2, marg, verbose=False):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    draw_params = dict(matchColor=(255,0,255), singlePointColor=(0,255,0), matchesMask=None, flags=cv2.DrawMatchesFlags_DEFAULT)
    img_match = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, **draw_params)

    x1 = np.stack([kp1[match[0].queryIdx].pt for match in good_matches],1)
    x2 = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)
    x1 = cv.homogenize(x1, multi=True)
    x2 = cv.homogenize(x2, multi=True)

    des1 = np.stack([des1[match[0].queryIdx] for match in good_matches],0)
    des2 = np.stack([des2[match[0].trainIdx] for match in good_matches],0)

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, kp1, kp2, des1, des2, img_match

def compute_sift_points_TR(x1, des1, img2, marg, verbose=False):
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    x_idx = np.array([match[0].queryIdx for match in good_matches])
    x1 = np.stack([x1[:,match[0].queryIdx] for match in good_matches],1)
    x2 = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)
    x2 = cv.homogenize(x2, multi=True)

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, x_idx

In [70]:
def compute_sift_points_vlfeat(img1, img2, marg, verbose=False):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) 
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) 

    sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # print('opencv:', des1.shape, des2.shape)

    # kp1, des1 = vl.sift.sift(np.array(img1, dtype=np.float32), compute_descriptor=True)
    # kp2, des2 = vl.sift.sift(np.array(img2, dtype=np.float32), compute_descriptor=True)

    # print('vlfeat.', des1.shape, des2.shape)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    # x1 = np.column_stack([kp1[match[0].queryIdx,:2] for match in good_matches])
    # x2 = np.column_stack([kp2[match[0].trainIdx,:2] for match in good_matches])
    x1 = np.column_stack([kp1[match[0].queryIdx].pt for match in good_matches])
    x2 = np.column_stack([kp2[match[0].trainIdx].pt for match in good_matches])
    x1 = cv.homogenize(x1, multi=True)
    x2 = cv.homogenize(x2, multi=True)

    des1 = np.row_stack([des1[match[0].queryIdx] for match in good_matches])
    des2 = np.row_stack([des2[match[0].trainIdx] for match in good_matches])

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, kp1, kp2, des1, des2

def compute_sift_points_TR_vlfeat(x1, des1, img2, marg, verbose=False):
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.xfeatures2d.SIFT_create()
    kp2, des2 = sift.detectAndCompute(img2, None)
    # print('opencv:', des2.shape)

    # kp2, des2 = vl.sift.sift(np.array(img2, dtype=np.float32), compute_descriptor=True)
    # print('vlfeat.', des2.shape)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    x_idx = np.array([match[0].queryIdx for match in good_matches])
    x1 = np.column_stack([x1[:,match[0].queryIdx] for match in good_matches])
    # x2 = np.column_stack([kp2[match[0].trainIdx,:2] for match in good_matches])
    x2 = np.column_stack([kp2[match[0].trainIdx].pt for match in good_matches])
    x2 = cv.homogenize(x2, multi=True)

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, x_idx

In [71]:
data_set = 1
K, img_names, init_pair, pixel_threshold = get_dataset_info(data_set)
K_inv = LA.inv(K)
imgs = cv.load_image(img_names, multi=True)
n_imgs = imgs.shape[0]
n_camera_pairs = n_imgs - 1
img1_init = imgs[init_pair[0]]
img2_init = imgs[init_pair[1]]

In [119]:
print('\n\n\n### Computing and saving SIFT-points ###\n')
print('## For rotation averaging ##')

sift_RA = False
marg = 0.75

if sift_RA:
    
    for i in range(n_camera_pairs):
        print("\nCamera pair:", i+1, "/", n_camera_pairs)
        img1 = imgs[i]
        img2 = imgs[i+1]
        x1, x2, _, _, _, _ = compute_sift_points_vlfeat(img1, img2, marg, verbose=True)
        np.save('data/dataset_{}_RA_x1_{}.npy'.format(data_set, i), x1)
        np.save('data/dataset_{}_RA_x2_{}.npy'.format(data_set, i), x2)



print('\n\n## For translation registration ##')

sift_TR = True
marg = 0.75

if sift_TR:
    
    print("\nImage:", init_pair[1]+1, "/", n_imgs, '(initial image pair)')
    x1_init, x2_init, kp1_init, kp2_init, des1_init, des2_init = compute_sift_points_vlfeat(img1_init, img2_init, marg, verbose=True)
    np.save('data/dataset_{}_TR_x1_{}.npy'.format(data_set, init_pair[1]), x1_init)
    np.save('data/dataset_{}_TR_x2_{}.npy'.format(data_set, init_pair[1]), x2_init)

    X_visible = np.ones(x1_init.shape[1], dtype=bool)
    np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, init_pair[0]), X_visible)
    np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, init_pair[1]), X_visible)

    for i in range(n_imgs):

        if i != init_pair[0] and i != init_pair[1]:
            
            print("\nImage:", i+1, "/", n_imgs)
            img2 = imgs[i]
            x1, x2, x_idx = compute_sift_points_TR_vlfeat(x1_init, des1_init, img2, marg, verbose=True)
            x_idx = np.sort(x_idx)
            X_visible = []
            t = 0

            for j in range(x1_init.shape[1]):
                try:
                    if j == x_idx[t]:
                        X_visible.append(True)
                        t += 1
                    else:
                        X_visible.append(False)
                except:
                    X_visible.append(False)
            X_visible = np.array(X_visible)
                    
            np.save('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, i), X_visible)
            np.save('data/dataset_{}_TR_x1_{}.npy'.format(data_set, i), x1)
            np.save('data/dataset_{}_TR_x2_{}.npy'.format(data_set, i), x2)




### Computing and saving SIFT-points ###

## For rotation averaging ##


## For translation registration ##

Image: 9 / 9 (initial image pair)
Number of matches: 13403
Number of good matches: 2666

Image: 2 / 9
Number of matches: 2666
Number of good matches: 1958

Image: 3 / 9
Number of matches: 2666
Number of good matches: 2009

Image: 4 / 9
Number of matches: 2666
Number of good matches: 1924

Image: 5 / 9
Number of matches: 2666
Number of good matches: 1975

Image: 6 / 9
Number of matches: 2666
Number of good matches: 1941

Image: 7 / 9
Number of matches: 2666
Number of good matches: 1945

Image: 8 / 9
Number of matches: 2666
Number of good matches: 1848


In [159]:
print('\n\n\n### Loading SIFT-points ###\n')

# SIFT points for rotation averaging
x1s_norm_RA = []
x2s_norm_RA = []
matlab = True

for i in range(n_camera_pairs):

    if matlab:
        x1 = cv.convert_mat_to_np('data/dataset_{}_RA_mat_x1_{}.mat'.format(data_set, i), 'x1')
        x2 = cv.convert_mat_to_np('data/dataset_{}_RA_mat_x2_{}.mat'.format(data_set, i), 'x2')
    else:
        x1 = np.load('data/dataset_{}_RA_x1_{}.npy'.format(data_set, i))
        x2 = np.load('data/dataset_{}_RA_x2_{}.npy'.format(data_set, i))

    x1_norm = cv.dehomogenize(K_inv @ x1)
    x2_norm = cv.dehomogenize(K_inv @ x2)
    x1s_norm_RA.append(x1_norm)
    x2s_norm_RA.append(x2_norm)
    
    
# SIFT points for translation registration
Xs_visible_TR = []
x1s_norm_TR = []
x2s_norm_TR = []

for i in range(n_imgs):

    if i != init_pair[0]:

        if matlab:
            x1 = cv.convert_mat_to_np('data/dataset_{}_TR_mat_x1_{}.mat'.format(data_set, i), 'x1')
            x2 = cv.convert_mat_to_np('data/dataset_{}_TR_mat_x2_{}.mat'.format(data_set, i), 'x2')
        else:
            x1 = np.load('data/dataset_{}_TR_x1_{}.npy'.format(data_set, i))
            x2 = np.load('data/dataset_{}_TR_x2_{}.npy'.format(data_set, i))

        x1_norm = cv.dehomogenize(K_inv @ x1)
        x2_norm = cv.dehomogenize(K_inv @ x2)
        x1s_norm_TR.append(x1_norm)
        x2s_norm_TR.append(x2_norm)

    elif i == init_pair[0]:
        x1s_norm_TR.append(None)
        x2s_norm_TR.append(None)

    if matlab:
        X_visible = cv.convert_mat_to_np('data/dataset_{}_TR_mat_X_visible_{}.mat'.format(data_set, i), 'X_visible')[0]
    else:
        X_visible = np.load('data/dataset_{}_TR_X_visible_{}.npy'.format(data_set, i))
        
    Xs_visible_TR.append(np.array(X_visible, dtype=bool))




### Loading SIFT-points ###



In [143]:
print('\n\n\n### Computing rotation averaging ###\n')

min_its = 5000
max_its = 50000
scale_its = 4
alpha = 0.99
P1 = cv.get_canonical_camera()
rots = [P1[:,:-1]]
np.save('data/dataset_{}_RA_R_{}.npy'.format(data_set, 0), rots[0])
# abs_trans = [P1[:,-1]]

for i in range(n_camera_pairs):    

    print('\nCamera pair:', i+1, '/', n_camera_pairs)
    
    x1_norm = x1s_norm_RA[i]
    x2_norm = x2s_norm_RA[i]
    E, inliers = estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold, verbose=True)

    np.save('data/dataset_{}_RA_E_{}.npy'.format(data_set, i), E)
    np.save('data/dataset_{}_RA_inliers_{}.npy'.format(data_set, i), inliers)

    x1_norm_inliers = x1_norm[:,inliers]
    x2_norm_inliers = x2_norm[:,inliers]

    P2_arr = cv.extract_P_from_E(E)
    X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
    P2, _ = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)

    R1 = rots[i]
    R2 = P2[:,:-1] / P2[-1,2]
    if LA.det(R2) < 0:
        R2 = -R2
    R = R2 @ R1
    rots.append(R)
    
    np.save('data/dataset_{}_RA_R_{}.npy'.format(data_set, i+1), R)
    print('Det(R):', LA.det(R))

    # T1 = abs_trans[i]
    # T2 = P2[:,-1] + (R2 @ R1.T @ T1)
    # abs_trans.append(T2)




### Computing rotation averaging ###


Camera pair: 1 / 8
Iteration: 1 T_E: 50000 T_H: 50000 epsilon_E: 0.05 epsilon_H: 0 No. inliers: 336 From: E 8-point alg.
Iteration: 5 T_E: 50000 T_H: 50000 epsilon_E: 0.06 epsilon_H: 0 No. inliers: 433 From: E 8-point alg.
Iteration: 5 T_E: 50000 T_H: 50000 epsilon_E: 0.11 epsilon_H: 0.0 No. inliers: 752 From: H 4-point alg.
Iteration: 23 T_E: 50000 T_H: 50000 epsilon_E: 0.12 epsilon_H: 0.01 No. inliers: 823 From: H 4-point alg.
Iteration: 26 T_E: 50000 T_H: 50000 epsilon_E: 0.32 epsilon_H: 0.01 No. inliers: 2140 From: E 8-point alg.
Iteration: 2456 T_E: 5340.0 T_H: 50000 epsilon_E: 0.49 epsilon_H: 0.01 No. inliers: 3308 From: E 8-point alg.
Iteration: 3756 T_E: 5000 T_H: 5000 epsilon_E: 0.5 epsilon_H: 0.32 No. inliers: 3389 From: H 4-point alg.
Iteration: 4772 T_E: 5000 T_H: 5000 epsilon_E: 0.53 epsilon_H: 0.32 No. inliers: 3574 From: E 8-point alg.
Bailout at iteration: 5000
No. valid coords for each camera pair: [3574 3574 7148    0]
Argmax(

In [144]:
rots = []
for i in range(n_imgs):
    R = np.load('data/dataset_{}_RA_R_{}.npy'.format(data_set, i))  
    rots.append(R)
    print(i, 'Det(R):', LA.det(R))

0 Det(R): 1.0
1 Det(R): 1.0125123842599335
2 Det(R): 1.0175945879825792
3 Det(R): 1.0209722695360859
4 Det(R): 1.025701289816967
5 Det(R): 1.031938521468865
6 Det(R): 1.0356941606075958
7 Det(R): 1.0469515650087073
8 Det(R): 1.0492952732766736


In [145]:
pair = 7
E = np.load('data/dataset_{}_RA_E_{}.npy'.format(data_set, pair))
inliers = np.load('data/dataset_{}_RA_inliers_{}.npy'.format(data_set, pair))
x1_norm = x1s_norm_RA[pair]
x2_norm = x2s_norm_RA[pair]

x1_norm_inliers = x1_norm
x2_norm_inliers = x2_norm

P1 = cv.get_canonical_camera()
P2_arr = cv.extract_P_from_E(E)
X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
P2, X = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)

R1 = rots[pair]
X = LA.inv(R1) @ X[:-1,:]

plot_3D_points(X[:,inliers])

No. valid coords for each camera pair: [ 6262  6262    68 12456]
Argmax(P2_arr): 3


In [146]:
print('\n\n\n### Computing inliers between initial_pair[0] and all other images ###\n')

min_its = 20000
max_its = 30000
scale_its = 1
alpha = 0.99
compute_inliers = True

if compute_inliers:
    
    for i in range(n_imgs):

        if i != init_pair[0]:
            print('\nImage:', i+1, '/', n_imgs)

            x1_norm = x1s_norm_TR[i]
            x2_norm = x2s_norm_TR[i]
            E, inliers = estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold, verbose=True)

            np.save('data/dataset_{}_TR_E_{}.npy'.format(data_set, i), E)
            np.save('data/dataset_{}_TR_E_inliers_{}.npy'.format(data_set, i), inliers)

            if i == init_pair[1]:
                # np.save('data/dataset_{}_TR_E_{}.npy'.format(data_set, init_pair[0]), E)
                np.save('data/dataset_{}_TR_E_inliers_{}.npy'.format(data_set, init_pair[0]), inliers)




### Computing inliers between initial_pair[0] and all other images ###


Image: 2 / 9
Iteration: 1 T_E: 30000 T_H: 30000 epsilon_E: 0.03 epsilon_H: 0 No. inliers: 58 From: E 8-point alg.
Iteration: 2 T_E: 30000 T_H: 30000 epsilon_E: 0.14 epsilon_H: 0.01 No. inliers: 242 From: H 4-point alg.
Iteration: 40 T_E: 30000 T_H: 30000 epsilon_E: 0.28 epsilon_H: 0.02 No. inliers: 490 From: H 4-point alg.
Iteration: 265 T_E: 30000 T_H: 30000 epsilon_E: 0.3 epsilon_H: 0.02 No. inliers: 541 From: E 8-point alg.
Iteration: 2666 T_E: 20000 T_H: 30000 epsilon_E: 0.43 epsilon_H: 0.02 No. inliers: 758 From: E 8-point alg.
Iteration: 2804 T_E: 20000 T_H: 30000 epsilon_E: 0.48 epsilon_H: 0.06 No. inliers: 848 From: H 4-point alg.
Iteration: 5743 T_E: 20000 T_H: 30000 epsilon_E: 0.49 epsilon_H: 0.06 No. inliers: 875 From: E 8-point alg.
Iteration: 11774 T_E: 20000 T_H: 24173.0 epsilon_E: 0.63 epsilon_H: 0.12 No. inliers: 1127 From: H 4-point alg.
Bailout at iteration: 20000

Image: 3 / 9
Iteration: 1 T_

c:\Users\erikn\skola\EEN020-Computer-Vision\project\computer_vision.py:486: RuntimeWarning: divide by zero encountered in double_scalars
  T = scale * np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))


Iteration: 115 T_E: 20000 T_H: 30000 epsilon_E: 0.54 epsilon_H: 0.01 No. inliers: 926 From: H 4-point alg.
Bailout at iteration: 20000

Image: 5 / 9
Iteration: 1 T_E: 30000 T_H: 30000 epsilon_E: 0.04 epsilon_H: 0 No. inliers: 73 From: E 8-point alg.
Iteration: 6 T_E: 30000 T_H: 30000 epsilon_E: 0.05 epsilon_H: 0.0 No. inliers: 86 From: H 4-point alg.
Iteration: 13 T_E: 30000 T_H: 30000 epsilon_E: 0.07 epsilon_H: 0.0 No. inliers: 131 From: E 8-point alg.
Iteration: 26 T_E: 30000 T_H: 30000 epsilon_E: 0.1 epsilon_H: 0.01 No. inliers: 182 From: H 4-point alg.
Iteration: 92 T_E: 30000 T_H: 30000 epsilon_E: 0.14 epsilon_H: 0.01 No. inliers: 252 From: H 4-point alg.
Iteration: 712 T_E: 30000 T_H: 30000 epsilon_E: 0.24 epsilon_H: 0.01 No. inliers: 415 From: E 8-point alg.
Iteration: 1246 T_E: 30000 T_H: 30000 epsilon_E: 0.25 epsilon_H: 0.02 No. inliers: 443 From: H 4-point alg.
Iteration: 1436 T_E: 20000 T_H: 30000 epsilon_E: 0.44 epsilon_H: 0.03 No. inliers: 768 From: H 4-point alg.
Iteratio

In [147]:
print('\n\n\n### Computing initial 3D-points ###\n')

pair = init_pair[1]
# pair = 1
E = np.load('data/dataset_{}_TR_E_{}.npy'.format(data_set, pair))
inliers = np.load('data/dataset_{}_TR_E_inliers_{}.npy'.format(data_set, pair))
x1_norm = x1s_norm_TR[pair]
x2_norm = x2s_norm_TR[pair]

x1_norm_inliers = x1_norm
x2_norm_inliers = x2_norm

P1 = cv.get_canonical_camera()
P2_arr = cv.extract_P_from_E(E)
X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
P2, X = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)

R1_init = rots[init_pair[0]]
X = LA.inv(R1_init) @ X[:-1,:]

plot_3D_points(X[:,inliers])




### Computing initial 3D-points ###

No. valid coords for each camera pair: [2074 2074   10 4138]
Argmax(P2_arr): 3


In [149]:
def estimate_T_robust(K, R, X, x_norm, min_its, max_its, scale_its, alpha, err_threshold_px, verbose=False):
    
    err_threshold = err_threshold_px / K[0,0]
    best_T = None
    best_inliers = None
    best_epsilon = 0
    n_points = x_norm.shape[1]
    n_samples = 2
    ransac_its = max_its

    t = 0
    while t < ransac_its:
        t += 1

        rand_mask = np.random.choice(n_points, n_samples, replace=False)
        
        # T = cv.estimate_T_DLT_1(x_norm[:,rand_mask], verbose=False)
        T = cv.estimate_T_DLT_2(R, x_norm[:,rand_mask], verbose=False)

        x_norm_proj = cv.dehomogenize(R @ X + T[:,np.newaxis])
        distance_arr = cv.compute_point_point_distance(x_norm_proj, x_norm)
        inliers = distance_arr**2 < err_threshold**2
        n_inliers = np.sum(inliers)
        epsilon = n_inliers / n_points

        if epsilon > best_epsilon:
            best_T = np.copy(T)
            best_inliers = np.copy(inliers)
            best_epsilon = epsilon
            ransac_its = cv.compute_ransac_iterations(alpha, best_epsilon, n_samples, min_its, max_its, scale_its)
            if verbose:
                print('Iteration:', t, 'T:', ransac_its, 'epsilon:', np.round(best_epsilon, 2), 'No. inliers:', np.sum(inliers))
    
    print('Bailout at iteration:', t)
    return best_T, best_inliers

In [161]:
print('\n\n\n### Computing translation registrations ###\n')

# Is the constraint img < init_pair[0] necessary?
# Compute valid cheirality in E_robust
# Compute valid inliers in E_robust

min_its = 0
max_its = 30000
scale_its = 1
alpha = 0.99
P1 = cv.get_canonical_camera()
trans = [P1[:,-1]]
np.save('data/dataset_{}_TR_T_{}.npy'.format(data_set, i), trans[0])

for i in range(n_imgs):

    if i != 0:

        print('\nImage:', i+1, '/', n_imgs)

        inliers = np.load('data/dataset_{}_TR_E_inliers_{}.npy'.format(data_set, i))
        X_visible = Xs_visible_TR[i]
        # X_visible_inliers = X[:,X_visible][:,inliers]
        X_visible_inliers = X[:,X_visible]

        if i == init_pair[0]:
            x_norm = x1s_norm_TR[init_pair[1]]   
        else:
            x_norm = x2s_norm_TR[i]
            
        # x_norm_inliers = x_norm[:,inliers] 
        x_norm_inliers = x_norm 
        
        R = rots[i]

        print(x_norm_inliers.shape, X_visible_inliers.shape, X_visible.shape, X.shape, X[:,X_visible].shape, inliers.shape)

        if i+1 >= 7:
            max_its = 100000
        T, inliers = estimate_T_robust(K, R, X_visible_inliers, x_norm_inliers, min_its, max_its, scale_its, alpha, 5*pixel_threshold, verbose=True)
        trans.append(T)

        np.save('data/dataset_{}_TR_T_{}.npy'.format(data_set, i), T)
        np.save('data/dataset_{}_TR_T_inliers_{}.npy'.format(data_set, i), inliers)  




### Computing translation registrations ###


Image: 1 / 9
(3, 2074) (3, 2074) (2074,) (3, 2074) (3, 2074) (2074,)
Iteration: 8 T: 30000 epsilon: 0.0 No. inliers: 1
Iteration: 20 T: 30000 epsilon: 0.0 No. inliers: 2
Iteration: 59 T: 30000 epsilon: 0.0 No. inliers: 3
Iteration: 72 T: 30000 epsilon: 0.0 No. inliers: 4
Iteration: 157 T: 30000 epsilon: 0.0 No. inliers: 5
Iteration: 638 T: 30000 epsilon: 0.0 No. inliers: 6
Iteration: 4018 T: 30000 epsilon: 0.0 No. inliers: 7
Iteration: 5145 T: 30000 epsilon: 0.0 No. inliers: 8
Iteration: 9700 T: 30000 epsilon: 0.0 No. inliers: 9
Bailout at iteration: 30000

Image: 2 / 9
(3, 1779) (3, 1779) (2074,) (3, 2074) (3, 1779) (1779,)
Iteration: 3 T: 30000 epsilon: 0.0 No. inliers: 2
Iteration: 83 T: 30000 epsilon: 0.0 No. inliers: 4
Iteration: 405 T: 30000 epsilon: 0.0 No. inliers: 5
Iteration: 794 T: 30000 epsilon: 0.0 No. inliers: 7
Iteration: 4272 T: 30000 epsilon: 0.01 No. inliers: 9
Bailout at iteration: 30000

Image: 3 / 9
(3, 1761) (3, 176

In [ ]:
trans = []
for i in range(n_imgs):
    T = np.load('data/dataset_{}_TR_T_{}.npy'.format(data_set, i))
    inliers = np.load('data/dataset_{}_TR_T_inliers_{}.npy'.format(data_set, i))  
    trans.append(T)
